In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_datareader import data

In [46]:
start_date = '2018-01-01'
end_date = '2018-01-31'
portfolio_tickers = []

In [47]:
stocks = data.DataReader('AAPL', 'yahoo', start_date, end_date)

In [48]:
stocks

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2018-01-02,172.300003,169.259995,170.160004,172.259995,25555900,166.804016
2018-01-03,174.550003,171.960007,172.529999,172.229996,29517900,166.774963
2018-01-04,173.470001,172.080002,172.539993,173.029999,22434600,167.549622
2018-01-05,175.369995,173.050003,173.440002,175.000000,23660000,169.457214
2018-01-08,175.610001,173.929993,174.350006,174.350006,20567800,168.827820
2018-01-09,175.059998,173.410004,174.550003,174.330002,21584000,168.808456
2018-01-10,174.300003,173.000000,173.160004,174.289993,23959900,168.769714
2018-01-11,175.490005,174.490005,174.589996,175.279999,18667700,169.728378
2018-01-12,177.360001,175.649994,176.179993,177.089996,25418100,171.481064


In [49]:
stocks['Close']

Date
2018-01-02    172.259995
2018-01-03    172.229996
2018-01-04    173.029999
2018-01-05    175.000000
2018-01-08    174.350006
2018-01-09    174.330002
2018-01-10    174.289993
2018-01-11    175.279999
2018-01-12    177.089996
2018-01-16    176.190002
2018-01-17    179.100006
2018-01-18    179.259995
2018-01-19    178.460007
2018-01-22    177.000000
2018-01-23    177.039993
2018-01-24    174.220001
2018-01-25    171.110001
2018-01-26    171.509995
2018-01-29    167.960007
2018-01-30    166.970001
2018-01-31    167.429993
Name: Close, dtype: float64

In [50]:
stocks['log return'] = np.log(stocks['Close']).diff().dropna()
stocks['peske'] = np.log(stocks['Close']/stocks['Close'].shift(1)).dropna()



In [51]:
stocks

,High,Low,Open,Close,Volume,Adj Close,log return,peske
Date,,,,,,,,
2018-01-02,172.300003,169.259995,170.160004,172.259995,25555900,166.804016,NaN,NaN
2018-01-03,174.550003,171.960007,172.529999,172.229996,29517900,166.774963,-0.000174,-0.000174
2018-01-04,173.470001,172.080002,172.539993,173.029999,22434600,167.549622,0.004634,0.004634
2018-01-05,175.369995,173.050003,173.440002,175.000000,23660000,169.457214,0.011321,0.011321
2018-01-08,175.610001,173.929993,174.350006,174.350006,20567800,168.827820,-0.003721,-0.003721
2018-01-09,175.059998,173.410004,174.550003,174.330002,21584000,168.808456,-0.000115,-0.000115
2018-01-10,174.300003,173.000000,173.160004,174.289993,23959900,168.769714,-0.000230,-0.000230
2018-01-11,175.490005,174.490005,174.589996,175.279999,18667700,169.728378,0.005664,0.005664
2018-01-12,177.360001,175.649994,176.179993,177.089996,25418100,171.481064,0.010273,0.010273


In [52]:
stocks['log return']


Date
2018-01-02         NaN
2018-01-03   -0.000174
2018-01-04    0.004634
2018-01-05    0.011321
2018-01-08   -0.003721
2018-01-09   -0.000115
2018-01-10   -0.000230
2018-01-11    0.005664
2018-01-12    0.010273
2018-01-16   -0.005095
2018-01-17    0.016381
2018-01-18    0.000893
2018-01-19   -0.004473
2018-01-22   -0.008215
2018-01-23    0.000226
2018-01-24   -0.016057
2018-01-25   -0.018012
2018-01-26    0.002335
2018-01-29   -0.020916
2018-01-30   -0.005912
2018-01-31    0.002751
Name: log return, dtype: float64

In [53]:
stocks = stocks[['Close','log return','peske']]

In [54]:
stocks

,Close,log return,peske
Date,,,
2018-01-02,172.259995,NaN,NaN
2018-01-03,172.229996,-0.000174,-0.000174
2018-01-04,173.029999,0.004634,0.004634
2018-01-05,175.000000,0.011321,0.011321
2018-01-08,174.350006,-0.003721,-0.003721
2018-01-09,174.330002,-0.000115,-0.000115
2018-01-10,174.289993,-0.000230,-0.000230
2018-01-11,175.279999,0.005664,0.005664
2018-01-12,177.089996,0.010273,0.010273


In [55]:
stocks = stocks.dropna()

In [56]:
stocks

,Close,log return,peske
Date,,,
2018-01-03,172.229996,-0.000174,-0.000174
2018-01-04,173.029999,0.004634,0.004634
2018-01-05,175.000000,0.011321,0.011321
2018-01-08,174.350006,-0.003721,-0.003721
2018-01-09,174.330002,-0.000115,-0.000115
2018-01-10,174.289993,-0.000230,-0.000230
2018-01-11,175.279999,0.005664,0.005664
2018-01-12,177.089996,0.010273,0.010273
2018-01-16,176.190002,-0.005095,-0.005095


In [57]:
stocks = stocks.reset_index(drop=True)

In [58]:
stocks

,Close,log return,peske
0,172.229996,-0.000174,-0.000174
1,173.029999,0.004634,0.004634
2,175.000000,0.011321,0.011321
3,174.350006,-0.003721,-0.003721
4,174.330002,-0.000115,-0.000115
5,174.289993,-0.000230,-0.000230
6,175.279999,0.005664,0.005664
7,177.089996,0.010273,0.010273
8,176.190002,-0.005095,-0.005095
9,179.100006,0.016381,0.016381
